# Dependencies 

In [1]:
# Dependencies: 
import os
import pandas as pd
import numpy as np 
import pickle
import time
import pymongo
from bs4 import BeautifulSoup
from splinter import Browser 
import requests
from pprint import pprint

# Portland MLS 
    -Navigate to homepage
    -Collect the number of pages and number of listing per page
    -Scrape the href to all the listings shown

In [5]:
# Initialize Splinter
executable_path = {"executable_path": "C:/Users/Tristan/chromedriver_win32/chromedriver"}
browser = Browser('chrome', **executable_path, headless=True)

# Directing splinter to webpage
portlandMLS_url = 'https://www.portlandmlsdirect.com/cgi-bin/real?pge=newsearch&state=na&widget=true&sortby=price&area=Portland&price_lo=0&price_hi=100000000&tot_bed_lo=0&tot_bath_lo=0&htype=ALL'
browser.visit(portlandMLS_url)

In [6]:
# Create variables
tracker = 0
link_list = []

# Create BeautifulSoup object
soupy = BeautifulSoup(browser.html, 'html.parser')

# Identify the number of pages of houses
number_of_pages = soupy.find(id='ia_btn_text').text
page_count = int(number_of_pages[5:])
print(page_count)

# Look going through all the pages of listings
for p in range(1, page_count):
    tracker += 1 
    
    # Identify all items in the gridview class
    viewgrid_all = soupy.find_all('div', class_='viewgrid')
    last_item = viewgrid_all[-1].get('id')
    viewgrid_count = int(last_item[8:]) + 1
    
    # Loop going through each home listing on a page
    for g in range(1, viewgrid_count):
        
        # Create BeautifulSoup object
        soupy = BeautifulSoup(browser.html, 'html.parser')
        
        # Click on each grid item
        elems = soupy.find('div', {'id':(f'viewgrid{g}')}).find('a')
        start_html = "https://www.portlandmlsdirect.com/"  
        link = start_html + elems['href']
        link_list.append(link)
    
    # Go to next page
    browser.find_by_id("ia_btn_next").click()
    print(f'Completion Percentage: {round((tracker/page_count)*100, 1)}%')
    
print(len(link_list))
print(link_list)

220
Completion Percentage: 0.5%
Completion Percentage: 0.9%
Completion Percentage: 1.4%
Completion Percentage: 1.8%
Completion Percentage: 2.3%
Completion Percentage: 2.7%
Completion Percentage: 3.2%
Completion Percentage: 3.6%
Completion Percentage: 4.1%
Completion Percentage: 4.5%
Completion Percentage: 5.0%
Completion Percentage: 5.5%
Completion Percentage: 5.9%
Completion Percentage: 6.4%
Completion Percentage: 6.8%
Completion Percentage: 7.3%
Completion Percentage: 7.7%
Completion Percentage: 8.2%
Completion Percentage: 8.6%
Completion Percentage: 9.1%
Completion Percentage: 9.5%
Completion Percentage: 10.0%
Completion Percentage: 10.5%
Completion Percentage: 10.9%
Completion Percentage: 11.4%
Completion Percentage: 11.8%
Completion Percentage: 12.3%
Completion Percentage: 12.7%
Completion Percentage: 13.2%
Completion Percentage: 13.6%
Completion Percentage: 14.1%
Completion Percentage: 14.5%
Completion Percentage: 15.0%
Completion Percentage: 15.5%
Completion Percentage: 15.9%
Co

In [10]:
# Check all links are unique
link_check = pd.DataFrame(link_list)
print(len(link_check[0].unique()))
print(len(link_check[0]))
link_list = list(link_check[0])

7882
7884


# Scrape Housing Data for each listing
    -address
    -price
    -bedrooms
    -bathrooms
    -square footage
    -year built
    -lot size
    -neighborhood
    -schools (elementary, middle and high)
    -county
    -city
    -zipcode

In [11]:
# Define variables
list_home_dict = []
loading_error_links = []
error_count = 0
bath = "BATH"
acres = "ACRES"
neighbor = "Neighborhood"

# Loop to iterate through all listing links
for links in link_list: 
    try:
        # Navigate to webpages for each home
        browser.visit(links)
        soup = BeautifulSoup(browser.html, 'html.parser')

        # To categorize homes with 0 bedrooms
        if bath in soup.find_all('div', class_="lineitem")[0].text:
            # Scraping primary data 
            address = soup.find('div', id="ia_address").text.replace("\n","").replace("\t  ", "")
            price = int(soup.find('div', id="ia_price").text.strip().replace('$', '').replace(',',''))
            bedrooms = 0 
            bathrooms = int(soup.find_all('div', class_="lineitem")[0].text[0])
            square_feet = int(soup.find_all('div', class_="lineitem")[1].text.replace(
                                                            'SQFT', '').replace(',', ''))
        
        else:
            # Scraping primary data
            address = soup.find('div', id="ia_address").text.replace("\n","").replace("\t  ", "")
            price = int(soup.find('div', id="ia_price").text.strip().replace('$', '').replace(',',''))
            bedrooms = int(soup.find_all('div', class_="lineitem")[0].text[0])
            bathrooms = float(soup.find_all('div', class_="lineitem")[1].text[0])
            square_feet = int(soup.find_all('div', class_="lineitem")[2].text.replace(
                                                            'SQFT', '').replace(',', ''))
        
        home_type = soup.find_all('div', id="PropDetailItem")[5].text[12:].replace('\n', '')
        built = int(soup.find_all('div', id="PropDetailItem")[3].text[12:])
        
        # Deals with pages that don't mention lot size
        if acres in soup.find_all('div', class_="lineitem")[4].text:
            lot_size = float(soup.find_all('div', class_="lineitem")[4].text.replace('ACRES', ''))

        else:
            lot_size = np.NaN 

        # Deals with pages that don't mention neighborhood
        if neighbor == soup.find_all('div', id='areaitemTitle')[3].text:
            neighborhood = soup.find_all('div', id='areaitemValue')[3].text

        else:
            neighborhood = "unknown"
            
        county = soup.find_all('div', id='areaitemValue')[0].text
        city = soup.find_all('div', id='areaitemValue')[1].text
        zipcode = soup.find_all('div', id='areaitemValue')[2].text

        # Schools data
        HS = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[10].text
        MS = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[6].text
        ES = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[2].text

        # Create dictionary to hold data collected
        home_dict = {
                'address':address,
                'price':price,
                'home_type':home_type,
                'bedrooms':bedrooms,
                'bathrooms':bathrooms,
                'square_feet':square_feet,
                'built':built,
                'lot_size':lot_size,
                'neighborhood':neighborhood,
                'county':county,
                'city':city,
                'zipcode':zipcode,
                'high_school':HS,
                'middle_school':MS,
                'elementary_school':ES
            }
        print(home_dict)

        # Append dictionaries to a list
        list_home_dict.append(home_dict)
    
    # Handles errors and collect links which data was able to be pulled
    except Exception as e:
        print('-------------')
        loading_error_links.append(links)
        error_count += 1
        print(error_count, e)
        
        pass

browser.quit()

{'address': '3223 SPRING VILLA ST, Silverton OR 97381', 'price': 250000, 'home_type': 'Manufactured - 1 Story', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1056, 'built': 1973, 'lot_size': 5.0, 'neighborhood': 'unknown', 'county': 'Marion', 'city': 'Silverton', 'zipcode': '97381', 'high_school': 'Silverton', 'middle_school': 'Silver Crest', 'elementary_school': 'Silver Crest'}
{'address': '42790 MYRTLE LN, Port Orford OR 97465', 'price': 250000, 'home_type': 'Single Family - Daylight Ranch', 'bedrooms': 3, 'bathrooms': 2.0, 'square_feet': 1836, 'built': 2011, 'lot_size': 0.39, 'neighborhood': 'unknown', 'county': 'Curry', 'city': 'Port Orford', 'zipcode': '97465', 'high_school': 'Pacific', 'middle_school': 'Driftwood', 'elementary_school': 'Driftwood'}
{'address': '47696 BEAVER ST, Oakridge OR 97463', 'price': 250000, 'home_type': 'Single Family - Ranch', 'bedrooms': 3, 'bathrooms': 2.0, 'square_feet': 1728, 'built': 2008, 'lot_size': 0.16, 'neighborhood': 'unknown', 'county': 'Lan

{'address': '1719 ARCH LN, Brookings OR 97415', 'price': 252000, 'home_type': 'Condo - Common Wall', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1424, 'built': 1987, 'lot_size': nan, 'neighborhood': 'E Harris Heights', 'county': 'Curry', 'city': 'Brookings', 'zipcode': '97415', 'high_school': 'Brookings-Harbr', 'middle_school': 'Azalea', 'elementary_school': 'Kalmiopsis'}
{'address': '8712 N DECATUR ST 301, Portland OR 97203', 'price': 252000, 'home_type': 'Condo - Contemporary', 'bedrooms': 2, 'bathrooms': 1.0, 'square_feet': 934, 'built': 2007, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97203', 'high_school': 'Roosevelt', 'middle_school': 'George', 'elementary_school': 'James John'}
{'address': '111 SW HARRISON ST 7G, Portland OR 97201', 'price': 253000, 'home_type': 'Condo - Contemporary', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 600, 'built': 1965, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', '

-------------
1 list index out of range
{'address': '324 JONATHAN LN, Florence OR 97439', 'price': 255000, 'home_type': 'Manufactured - Double Wide Manufact', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1345, 'built': 1992, 'lot_size': 0.15, 'neighborhood': 'FLORENTINE ESTATES', 'county': 'Lane', 'city': 'Florence', 'zipcode': '97439', 'high_school': 'Siuslaw', 'middle_school': 'Siuslaw', 'elementary_school': 'Siuslaw'}
{'address': '760 NW 185TH AVE 307, Beaverton OR 97006', 'price': 255000, 'home_type': 'Condo - Traditional', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1007, 'built': 2007, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Washington', 'city': 'Beaverton', 'zipcode': '97006', 'high_school': 'Westview', 'middle_school': 'Five Oaks', 'elementary_school': 'Elmonica'}
{'address': '545 S WILLIAMS ST, Lebanon OR 97355', 'price': 255000, 'home_type': 'Single Family - 2 Story', 'bedrooms': 4, 'bathrooms': 2.0, 'square_feet': 1969, 'built': 1938, 'lot_size': 0.2, '

{'address': '1500 SW SKYLINE BLVD 16, Portland OR 97221', 'price': 258900, 'home_type': 'Condo - Common Wall', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1156, 'built': 1966, 'lot_size': nan, 'neighborhood': 'Sylvan/Skyline', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97221', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Ainsworth'}
{'address': '1416 1/2 E 10TH PL, The Dalles OR 97058', 'price': 259000, 'home_type': 'Single Family - Cottage', 'bedrooms': 4, 'bathrooms': 2.0, 'square_feet': 1896, 'built': 1915, 'lot_size': 0.11, 'neighborhood': 'unknown', 'county': 'Wasco', 'city': 'The Dalles', 'zipcode': '97058', 'high_school': 'The Dalles-Waht', 'middle_school': 'The Dalles', 'elementary_school': 'Dry Hollow'}
{'address': '54973 ROSA RD, Bandon OR 97411', 'price': 259000, 'home_type': 'Single Family - Traditional', 'bedrooms': 3, 'bathrooms': 1.0, 'square_feet': 1456, 'built': 1940, 'lot_size': 6.71, 'neighborhood': 'unknown', 'count

{'address': '2290 NW JETTY AVE, Lincoln City OR 97367', 'price': 259500, 'home_type': 'Single Family - Chalet', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1348, 'built': 1963, 'lot_size': 0.17, 'neighborhood': 'unknown', 'county': 'Lincoln', 'city': 'Lincoln City', 'zipcode': '97367', 'high_school': 'Taft', 'middle_school': 'Taft', 'elementary_school': 'Oceanlake'}
{'address': '17534 SE MARIE ST, Portland OR 97236', 'price': 259800, 'home_type': 'Single Family - 1 Story', 'bedrooms': 2, 'bathrooms': 1.0, 'square_feet': 763, 'built': 1926, 'lot_size': 0.18, 'neighborhood': 'Centennial', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97236', 'high_school': 'Centennial', 'middle_school': 'Centennial', 'elementary_school': 'Meadows'}
{'address': '255 SW HARRISON ST 15b, Portland OR 97201', 'price': 259800, 'home_type': 'Condo - Contemporary', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 735, 'built': 1965, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', '

...

{'address': '17416 SW ROOSEVELT ST, Sherwood OR 97140', 'price': 519900, 'home_type': 'Single Family - 2 Story', 'bedrooms': 4, 'bathrooms': 3.0, 'square_feet': 2242, 'built': 2004, 'lot_size': 0.16, 'neighborhood': 'Heron Ridge. Estates', 'county': 'Washington', 'city': 'Sherwood', 'zipcode': '97140', 'high_school': 'Sherwood', 'middle_school': 'Sherwood', 'elementary_school': 'Edy Ridge'}
{'address': '2205 S GIOVANNI AVE, Salem OR 97306', 'price': 519900, 'home_type': 'Single Family - 2 Story', 'bedrooms': 3, 'bathrooms': 2.0, 'square_feet': 2796, 'built': 2018, 'lot_size': 0.26, 'neighborhood': 'Verona Heights #2', 'county': 'Marion', 'city': 'Salem', 'zipcode': '97306', 'high_school': 'Sprague', 'middle_school': 'Crossler', 'elementary_school': 'Liberty'}
{'address': '3787 SW 31ST ST, Gresham OR 97080', 'price': 519950, 'home_type': 'Single Family - 2 Story', 'bedrooms': 4, 'bathrooms': 3.0, 'square_feet': 2589, 'built': 2020, 'lot_size': 0.11, 'neighborhood': 'unknown', 'county': 

{'address': '1591 NE 17th AVE, Canby OR 97013', 'price': 523900, 'home_type': 'Single Family - Farmhouse', 'bedrooms': 4, 'bathrooms': 3.0, 'square_feet': 2287, 'built': 2020, 'lot_size': 0.14, 'neighborhood': 'Redwood Landing', 'county': 'Clackamas', 'city': 'Canby', 'zipcode': '97013', 'high_school': 'Canby', 'middle_school': 'Baker Prairie', 'elementary_school': 'Knight'}
{'address': '13733 SE LUCA AVE, Clackamas OR 97015', 'price': 524000, 'home_type': 'Condo - Craftsman', 'bedrooms': 4, 'bathrooms': 4.0, 'square_feet': 3034, 'built': 2008, 'lot_size': 0.11, 'neighborhood': 'Ponte Cino', 'county': 'Clackamas', 'city': 'Clackamas', 'zipcode': '97015', 'high_school': 'Clackamas', 'middle_school': 'Rock Creek', 'elementary_school': 'Sunnyside'}
{'address': '3519 NW 372ND ST, La Center WA 98629', 'price': 524000, 'home_type': 'Single Family - Ranch', 'bedrooms': 3, 'bathrooms': 2.0, 'square_feet': 1652, 'built': 1994, 'lot_size': 5.74, 'neighborhood': 'unknown', 'county': 'Clark', 'cit

{'address': '910 SW 213th TER VL 09, Beaverton OR 97003', 'price': 524950, 'home_type': 'Single Family - 2 Story', 'bedrooms': 3, 'bathrooms': 3.0, 'square_feet': 2222, 'built': 2020, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Washington', 'city': 'Beaverton', 'zipcode': '97003', 'high_school': 'Century', 'middle_school': 'Brown', 'elementary_school': 'Tobias'}
{'address': '16908 SE HUCKLEBERRY ST, Happy Valley OR 97086', 'price': 524950, 'home_type': 'Single Family - Contemporary', 'bedrooms': 4, 'bathrooms': 3.0, 'square_feet': 2593, 'built': 2020, 'lot_size': 0.1, 'neighborhood': 'unknown', 'county': 'Clackamas', 'city': 'Happy Valley', 'zipcode': '97086', 'high_school': 'Centennial', 'middle_school': 'Centennial', 'elementary_school': 'Pleasant Valley'}
{'address': '10040 SE ELDERBERRY ST, Happy Valley OR 97086', 'price': 524950, 'home_type': 'Single Family - Contemporary', 'bedrooms': 4, 'bathrooms': 3.0, 'square_feet': 2633, 'built': 2019, 'lot_size': 0.09, 'neighborh

{'address': '310 BATTLE MOUNTAIN LN, Glendale OR 97442', 'price': 525000, 'home_type': 'Single Family - Chalet', 'bedrooms': 4, 'bathrooms': 4.0, 'square_feet': 3786, 'built': 1985, 'lot_size': 11.24, 'neighborhood': 'unknown', 'county': 'Douglas', 'city': 'Glendale', 'zipcode': '97442', 'high_school': 'Glendale', 'middle_school': 'Glendale', 'elementary_school': 'Glendale'}
{'address': '833 RIVER LOOP 1 , Eugene OR 97404', 'price': 525000, 'home_type': 'Single Family - Mid-Century Modern', 'bedrooms': 4, 'bathrooms': 2.0, 'square_feet': 2156, 'built': 1960, 'lot_size': 1.94, 'neighborhood': 'unknown', 'county': 'Lane', 'city': 'Eugene', 'zipcode': '97404', 'high_school': 'North Eugene', 'middle_school': 'Madison', 'elementary_school': 'Awbrey Park'}
{'address': '3748 SW BRITTANY DR, Gresham OR 97080', 'price': 525000, 'home_type': 'Single Family - Ranch', 'bedrooms': 3, 'bathrooms': 2.0, 'square_feet': 2429, 'built': 1993, 'lot_size': 0.22, 'neighborhood': 'Hunters Highland', 'county'

KeyboardInterrupt: 

In [9]:
# Create a dataframe
housing_data_df = pd.DataFrame(list_home_dict)
housing_data_df.drop_duplicates(inplace=True)
housing_data_df.head()

,address,price,home_type,bedrooms,bathrooms,square_feet,built,lot_size,neighborhood,county,city,zipcode,high_school,middle_school,elementary_school
0,"3157 NE MARINE DR, Portland OR 97035",65000,Floating Home - 1 Story,1,1.0,800,1964,NaN,unknown,Multnomah,Portland,97035,Current Price:,Jefferson,Faubion
1,"17452 NE GLISAN ST #7, Portland OR 97230",72000,Manufactured - Double Wide Manufact,2,2.0,1152,1988,NaN,unknown,Multnomah,Portland,97230,Reynolds,Reynolds,Hartley
2,"9034 SE 78TH PL, Portland OR 97206",79950,Manufactured - Double Wide Manufact,3,2.0,1344,1997,NaN,unknown,Clackamas,Portland,97206,Current Price:,Milwaukie,Whitman
3,"16000 SE POWELL BLVD 75, Portland OR 97236",79950,Manufactured - Double Wide Manufact,3,2.0,1404,1990,NaN,unknown,Multnomah,Portland,97236,Centennial,Centennial,Powell Butte
4,"12846 SE RAMONA ST 6, Portland OR 97236",93900,Manufactured - Double Wide Manufact,3,2.0,1297,1997,NaN,unknown,Multnomah,Portland,97236,David Douglas,Alice Ott,Gilbert Hts


In [11]:
# Check data collected verse data missed
print(f'Listings: {len(housing_data_df)}')
print(f'Missed Listings: {len(loading_error_links)}')

Listings: 1781
Missed Listings: 55


# Save Data

In [13]:
# Saving housing data
housing_data_df.to_csv("../Resources/housingData.csv", index = False, header = True)

# Saving list of housing links
with open("../Resources/housing_links.txt", "wb") as fp:   #Pickling
    pickle.dump(link_list, fp)